In [8]:
import pickle
import pandas as pd

In [9]:
df = pd.read_pickle('turbine1_df_final.pkl')



In [10]:
from sklearn.model_selection import train_test_split

# Split the dataset into training, validation, and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=False)
train_df, val_df = train_test_split(train_df, test_size=0.25, random_state=42, shuffle=False)

# Print the shape of each split
print('Training set shape:', train_df.shape)
print('Validation set shape:', val_df.shape)
print('Testing set shape:', test_df.shape)

Training set shape: (78436, 63)
Validation set shape: (26146, 63)
Testing set shape: (26146, 63)


In [1]:
from pytorch_tabnet.tab_model import TabNetRegressor

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np
np.random.seed(0)


import os
